# Decision tree - migration destination choice

#### Imports

In [ ]:
# imports
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import tree
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import cluster_methods
import decisiontree_help
import cluster_vis

import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

#### Reading the files 
Choose between data with or without global region variables.

In [ ]:
# reading the data

source = "gwp_data/prepared_destination/clean_data_from18to22_"
df_w = pd.read_pickle(source)

In [ ]:
df_w.drop(columns=['WP1325: Move Permanently to Another Country'], axis=1, inplace=True)
df_w['WP3120: Country Would Move To'] = df_w['WP3120: Country Would Move To'].map(lambda x: 0 if x != 1 else 1)

In [ ]:
# Important characteristics of the dataframe
print("Size of the dataframe:", df_w.shape)

In [ ]:
with open('meta/countrynum_to_name_dict.pickle', 'rb') as fp:
    countrynum_to_name_dict = pickle.load(fp)

with open('meta/countrynum_to_ISO_dict.pickle', 'rb') as fp:
    countrynum_to_ISO_dict = pickle.load(fp)

In [ ]:
source = "meta/columns"
df_meta = pd.read_pickle(source)

In [ ]:
# save the original dataframe to be able to recover the answers
df_original_w = df_w.copy()

#### Data preparation for the decision tree

In [ ]:
# keep only predictive columns
non_pred_columns = ['Region: Region',\
                    'YEAR_WAVE: Wave Year',\
                    'COUNTRY_ISO3: Country ISO alpha-3 code',\
                    'WP1325: Move Permanently to Another Country',\
                    'WP3120: Country Would Move To',\
                    'WP5889: Questionnaire Serial Number'   ]

df_w = df_w.loc[:, ~df_w.columns.isin(non_pred_columns)]

In [ ]:
df_w['WP9048: Country Where Born'] = df_w['WP9048: Country Where Born']==df_w["WP5: Country"]
df_w.drop('WP5: Country', axis=1, inplace=True)

#### Categorical data

In [ ]:
yes_columns = []
yn_columns = []
ordinal_columns = []
no_columns = []

for col in df_w.columns:
    l = list(df_meta[df_meta['column'].str.contains(col)]["categorical?"])
    if len(l) !=0:
        if "yes" in l[0]:
            yes_columns.append(col)
        if "yn" in l[0] :
            yn_columns.append(col)
        if "ordinal" in l[0] :
            ordinal_columns.append(col)
        if "no" in l[0] :
            no_columns.append(col)

In [ ]:
cat_columns = set(yn_columns + yes_columns).intersection(df_w.columns)
cat_columns = cat_columns.difference(set('WP3120: Country Would Move To'))

df_w = pd.get_dummies(df_w, columns = cat_columns, prefix=cat_columns)
df_w.convert_dtypes()

cat_columns = set(yn_columns + yes_columns).intersection(df_w.columns)
cat_columns = cat_columns.difference(set('WP3120: Country Would Move To'))

df_w = pd.get_dummies(df_w, columns = cat_columns, prefix=cat_columns)
df_w.convert_dtypes()

In [ ]:
X_w = df_w.loc[:, ~df_w.columns.isin(['WP3120: Country Would Move To'])]
X_w.dropna(axis=1, inplace=True)

Y_w = df_original_w['WP3120: Country Would Move To'].astype(int)

# Without region

#### Training the decision tree

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X_w, Y_w)

plt.figure(figsize=(12,12), dpi=400)
tree.plot_tree(clf, fontsize=3, filled=True)

In [ ]:
sum(clf.predict(X_w) == Y_w) / len(Y_w)

In [ ]:
decisiontree_help.tree_to_code(clf, df_w.columns)

In [ ]:
tree_ = clf.tree_
tree_.feature

### Clustering 1 - MAX Leaves
The countries are in the same cluster if they are most represented in the same leafes.

In [ ]:
name_iso = df_original_w.get(["WP5: Country", "COUNTRY_ISO3: Country ISO alpha-3 code"])

In [ ]:
df_with_country = X_w.join(name_iso, how='left')
df_with_country

In [ ]:
df_cluster1 = decisiontree_help.clustering1_depth3(clf, X_w, df_with_country, countrynum_to_name_dict)
df_cluster1["WP5: Country"] = [countrynum_to_name_dict[c_code] for c_code in df_cluster1.index]
df_cluster1["COUNTRY_ISO3: Country ISO alpha-3 code"] = [countrynum_to_ISO_dict[c_code] for c_code in df_cluster1.index]
cluster_vis.cluster_visualization(df_cluster1, df_cluster1["cluster_1"], "D1")

In [ ]:
df_cluster1


### Clustering 2 - MAX 2 leaves
The countries are in the same cluster if the two most represented leafes are the same.

In [ ]:
df_cluster2 = decisiontree_help.clustering2(df_cluster1, 3, countrynum_to_name_dict)
df_cluster2["WP5: Country"] = [countrynum_to_name_dict[c_code] for c_code in df_cluster2.index]
df_cluster2["COUNTRY_ISO3: Country ISO alpha-3 code"] = [countrynum_to_ISO_dict[c_code] for c_code in df_cluster2.index]
cluster_vis.cluster_visualization(df_cluster2, df_cluster2["cluster_2"], "D2")

In [ ]:
df_cluster2

### Clustering 3 - DBSCAN

In [ ]:
decisiontree_help.run_clusters_distribution(4, X_w, Y_w, df_original_w, 'dbscan', 0.040) 

### Clustering 4 - K-Means

#### depth = 8

In [ ]:
df_help_8_kmeans = decisiontree_help.create_df(4, X_w, Y_w, df_original_w)
cluster_methods.elbow_method(df_help_8_kmeans, 2, 20, "kmeans")

In [ ]:
decisiontree_help.run_clusters_distribution(4, X_w, Y_w, df_original_w, 'kmeans', 5)

#### depth = max

In [ ]:
df_help_max_kmeans = decisiontree_help.create_df('max', X_w, Y_w, df_original_w)
cluster_methods.elbow_method(df_help_max_kmeans, 2, 10, "kmeans")

In [ ]:
decisiontree_help.run_clusters_distribution('max', X_w, Y_w, df_original_w, 'kmeans', 3) 

### Clustering 5 - Agglomerative clustering

#### depth = 8

In [ ]:
df_help_8_agglo = decisiontree_help.create_df(4, X_w, Y_w, df_original_w)
cluster_methods.elbow_method(df_help_8_agglo, 2, 20, "agglo")

In [ ]:
decisiontree_help.run_clusters_distribution(4, X_w, Y_w, df_original_w, 'agglo', 8) 

#### depth = max

In [ ]:
df_help_max_agglo = decisiontree_help.create_df('max', X_w, Y_w, df_original_w)
cluster_methods.elbow_method(df_help_max_agglo, 2, 20, "agglo")

In [ ]:
decisiontree_help.run_clusters_distribution('max', X_w, Y_w, df_original_w, 'agglo', 7) 